# 02) 정상/감염 이진분류 모델 학습(Inception v4)

In [ ]:
use_background = 1
use_agumentation = 1
use_dropout = 1
use_l2 = 1

## 라이브러리 설치 및 Import

In [ ]:
''' 패키지 설치 '''
!pip install torch torchvision

# =============================
# torch : 모델 실행, 학습, 추론에 필수적인 PyTorch 프레임워크
# torchvision : 이미지 처리 관련 도구 제공
# =============================

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
'''Google Drive 연동'''
from google.colab import drive
drive.mount('/content/drive')

'''필수 라이브러리 import'''
import pandas as pd
import numpy as np
import shutil
import zipfile
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F

Mounted at /content/drive


## 데이터 불러오기

In [ ]:
'''Label 데이터 불러오기'''
rawdata = pd.read_csv("/content/drive/MyDrive/data/rawdata.csv")

In [ ]:
'''Image 데이터 불러오기'''
if use_background:
  zipName = 'data'
else:
  zipName = 'raw'

# data 폴더 생성
targetPath = '/content/data/'
os.makedirs(targetPath, exist_ok=True)

# 압축 파일 content로 복사 => content에 있으면 처리속도가 비교적 빠름
rootZip = f'/content/drive/MyDrive/data/{zipName}.zip'
targetZip = f'/content/{zipName}.zip'
shutil.copyfile(rootZip, targetZip)

# zipfile.ZipFile로 압축 파일을 열고, 압축된 모든 파일을 targetPath로 이동(압축 해제)
with zipfile.ZipFile(targetZip, 'r') as zip_ref:
  zip_ref.extractall(targetPath)

## 데이터 Split

In [ ]:
''' Train:Val:Test = 6:2:2 분할'''
# rawdata를 분할. train:(val+test) = 6:4
train, temp = train_test_split(rawdata, test_size=0.4, random_state=1)

# rawdata를 분할. val:test = 5:5
val, test = train_test_split(temp, test_size=0.5, random_state=1)

## 데이터셋 클래스, Transform 생성

In [ ]:
''' DataSet 클래스 정의 '''
# =============================
# 목적 : DataFrame에 저장된 이미지 경로와 라벨 등을 불러오고, 전처리 후 (image, label) 리턴
# 매개변수(?)
#  - dataframe : 이미지 파일 이름/클래스 등이 포함된 변수
#  - rootDir : 이미지들이 저장된 경로
#  - transform : torchvision.transforms를 사용한 이미지 전처리 파이프라인(전처리 묶음? 정도로 이해하면 될 듯)
# =============================
class CustomDataset(Dataset):
  def __init__(self, dataframe, rootDir, transform=None):
    self.dataframe = dataframe
    self.rootDir = rootDir
    self.transform = transform

  # 데이터셋의 총 샘플 수 반환
  def __len__(self):
    return len(self.dataframe)

  # idx번째 데이터 리턴
  def __getitem__(self, idx):
    # DataFrame의 idx번째 행을 row에 저장
    row = self.dataframe.iloc[idx]
    # 이미지 경로 불러오기(row['image']는 파일명이므로, 상위 경로와 더해줌; 파일 이름이 '.JPG'로 된 경우도 있어서 통일)
    imgName = os.path.join(self.rootDir, row['image'].replace('.JPG', '.jpg'))

    # 이미지 파일을 열고, RGB로 변환
    image = Image.open(imgName).convert('RGB')

    # trasform이 정의되어 있다면, 전처리 적용
    if self.transform:
        image = self.transform(image)

    # 라벨 저장
    label = row['class']

    # (전처리된 이미지, 라벨) 리턴
    return image, label

In [ ]:
'''Transform 정의'''
# train, val, test 용도에 따라 전처리를 다르게 적용
if use_agumentation:
  transform = {
      'train': transforms.Compose([
          # 이미지를 299x299로 통일 (Inception v4 입력 크기)
          transforms.Resize((299, 299), interpolation=transforms.InterpolationMode.BICUBIC),
          # 확률적으로 이미지를 좌우 반전 (데이터 증강)
          transforms.RandomHorizontalFlip(),
          # -15 ~ +15도 사이로 랜덤 회전 (데이터 증강)
          transforms.RandomRotation(15),
          # 밝기, 대비, 채도 랜덤 조절 (데이터 증강)
          transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
          # 10 % 비율로 좌우 또는 상화 랜덤 이동 (데이터 증강)
          transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
          # PIL 이미지를 Tensor 형식으로 변환 (0~299 >> 0~1 실수형)
          transforms.ToTensor(),
          # 평균 및 표준편차를 기준으로 정규화 (Image Net으로 사전 학습도니 모델과 일치시키기 위해)
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ]),
      'val': transforms.Compose([
          # 이미지를 299x299로 통일 (Inception v4 입력 크기)
          transforms.Resize((299, 299), interpolation=transforms.InterpolationMode.BICUBIC),
          # PIL 이미지를 Tensor 형식으로 변환 (0~299 >> 0~1 실수형)
          transforms.ToTensor(),
          # 평균 및 표준편차를 기준으로 정규화 (Image Net으로사전 학습도니 모델과 일치시키기 위해)
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ]),
      'test': transforms.Compose([ # val과 동일/ train과 달리 평가 용도기 때문에 val과 test에는 데이터 증강이 없음.
          transforms.Resize((299, 299), interpolation=transforms.InterpolationMode.BICUBIC),
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ])
  }
else:
  transform = {
      'train': transforms.Compose([
          # 이미지를 299x299로 통일 (Inception v4 입력 크기)
          transforms.Resize((299, 299), interpolation=transforms.InterpolationMode.BICUBIC),
          # PIL 이미지를 Tensor 형식으로 변환 (0~299 >> 0~1 실수형)
          transforms.ToTensor(),
          # 평균 및 표준편차를 기준으로 정규화 (Image Net으로 사전 학습도니 모델과 일치시키기 위해)
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ]),
      'val': transforms.Compose([
          # 이미지를 299x299로 통일 (Inception v4 입력 크기)
          transforms.Resize((299, 299), interpolation=transforms.InterpolationMode.BICUBIC),
          # PIL 이미지를 Tensor 형식으로 변환 (0~299 >> 0~1 실수형)
          transforms.ToTensor(),
          # 평균 및 표준편차를 기준으로 정규화 (Image Net으로사전 학습도니 모델과 일치시키기 위해)
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ]),
      'test': transforms.Compose([ # val과 동일/ train과 달리 평가 용도기 때문에 val과 test에는 데이터 증강이 없음.
          transforms.Resize((299, 299), interpolation=transforms.InterpolationMode.BICUBIC),
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
      ])
  }

In [ ]:
# Dataset 불러오기
trainDataset = CustomDataset(train, '/content/data', transform=transform['train'])
valDataset = CustomDataset(val, '/content/data', transform=transform['val'])
testDataset = CustomDataset(test, '/content/data', transform=transform['test'])

# DataLoader 불러오기
trainLoader = DataLoader(trainDataset, batch_size=32, shuffle=False)
valLoader = DataLoader(valDataset, batch_size=32, shuffle=False)
testLoader = DataLoader(testDataset, batch_size=32, shuffle=False)

## 학습, 테스트 메소드 정의

In [ ]:
''' Inception-V4 모델 생성 및 정의 '''

# 사용할 디바이스 설정 : GPU(cuda)가 가능하면 GPU, 아니면 CPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# =============================
# 목적 : 모델을 불러오고 재정의
# =============================
def create_model():
    # 학습목적으로 Inception v4모델을 직접 만들거임
    # 타 모델과 차이점: 여러 크기의 필터를 병렬로 사용함 -> 다양한 관점?에서 특징 추출 가능함

    # 구조: Input -> STEM -> Inception-A -> Reduction-A -> I-B -> R-B -> I-C -> Global Average Pooling -> Dropout -> FC -> Softmax

    # Stem: 모델의 첫 부분으로, Input을 받아 특징 추출 및 차원 축소 역할
    # Inception: 다양한 필터를 병렬로 사용해 특징 추출. 뒷 문자가 커질수록 고수준의 특징 추출(복잡한 패턴)
    # Reduction: 이미지 해상도 줄임(다운샘플링). 다음 블록을 위한 연결 다리 역할
    # Global Average Pooling: 마지막 전체 특징을 압축해 1차원 벡터로 만듬
    # Dropout: 과적합 방지
    # FC: 최종 분류 출력

    # Stem의 출력은 항상 35x35임.
    class Stem(nn.Module):
      def __init__(self):
          super(Stem, self).__init__()
          self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2)  # 149x149
          self.conv2 = nn.Conv2d(32, 32, kernel_size=3)           # 147x147
          self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # 147x147

          self.branch1 = nn.MaxPool2d(kernel_size=3, stride=2)    # 73x73
          self.branch2 = nn.Conv2d(64, 96, kernel_size=3, stride=2)  # 73x73

          self.branch3a = nn.Sequential(
              nn.Conv2d(160, 64, kernel_size=1),
              nn.Conv2d(64, 96, kernel_size=3)
          )

          self.branch3b = nn.Sequential(
              nn.Conv2d(160, 64, kernel_size=1),
              nn.Conv2d(64, 64, kernel_size=(7, 1), padding=(3, 0)),
              nn.Conv2d(64, 64, kernel_size=(1, 7), padding=(0, 3)),
              nn.Conv2d(64, 96, kernel_size=3)
          )

          self.branch4a = nn.Conv2d(192, 192, kernel_size=3, stride=2)
          self.branch4b = nn.MaxPool2d(kernel_size=3, stride=2)

      def forward(self, x):
          x = self.conv1(x)
          x = self.conv2(x)
          x = self.conv3(x)
          x1 = self.branch1(x)
          x2 = self.branch2(x)
          x = torch.cat([x1, x2], 1)

          x1 = self.branch3a(x)
          x2 = self.branch3b(x)
          x = torch.cat([x1, x2], 1)

          x1 = self.branch4a(x)
          x2 = self.branch4b(x)
          x = torch.cat([x1, x2], 1)
          return x


    # InceptionA의 출력 채널 수는 항상 384임.
    class InceptionA(nn.Module):
      def __init__(self, in_channels):
        super(InceptionA, self).__init__()

        # 1x1 Conv
        self.branch1 = nn.Conv2d(in_channels, 96, kernel_size=1)

        # 1x1 Conv -> 3x3 Conv
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=1),
            nn.Conv2d(64, 96, kernel_size=3, padding=1)
        )

        # 1x1 Conv -> 3x3 Conv -> 3x3 Conv
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=1),
            nn.Conv2d(64, 96, kernel_size=3, padding=1),
            nn.Conv2d(96, 96, kernel_size=3, padding=1)
        )

        # Average Pooling -> 1x1 Conv
        self.branch4 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, 96, kernel_size=1)
        )
      def forward(self, x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        b4 = self.branch4(x)
        # 채널 방향으로 합치기
        out = torch.cat([b1, b2, b3, b4], dim=1)
        return out

    class ReductionA(nn.Module):
      def __init__(self, in_channels):
        super(ReductionA, self).__init__()

        self.branch1 = nn.Conv2d(in_channels, 384, kernel_size=3, stride=2)

        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, 192, kernel_size=1),
            nn.Conv2d(192, 224, kernel_size=3, padding=1),
            nn.Conv2d(224, 256, kernel_size=3, stride=2)
        )

        self.branch3 = nn.MaxPool2d(kernel_size=3, stride=2)

      def forward(self, x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        out = torch.cat([b1, b2, b3], dim=1)
        return out

    class InceptionB(nn.Module):
      def __init__(self, in_channels):
        super(InceptionB, self).__init__()

        self.branch1 = nn.Conv2d(in_channels, 384, kernel_size=1)

        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, 192, kernel_size=1),
            nn.Conv2d(192, 224, kernel_size=(1, 7), padding=(0, 3)),
            nn.Conv2d(224, 256, kernel_size=(7, 1), padding=(3, 0))
        )

        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, 192, kernel_size=1),
            nn.Conv2d(192, 192, kernel_size=(7, 1), padding=(3, 0)),
            nn.Conv2d(192, 224, kernel_size=(1, 7), padding=(0, 3)),
            nn.Conv2d(224, 224, kernel_size=(7, 1), padding=(3, 0)),
            nn.Conv2d(224, 256, kernel_size=(1, 7), padding=(0, 3))
        )

        self.branch4 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, 128, kernel_size=1)
        )

      def forward(self, x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        b4 = self.branch4(x)
        out = torch.cat([b1, b2, b3, b4], dim=1)
        return out

    class ReductionB(nn.Module):
      def __init__(self, in_channels):
        super(ReductionB, self).__init__()

        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, 192, kernel_size=1),
            nn.Conv2d(192, 192, kernel_size=3, stride=2)
        )

        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, 256, kernel_size=1),
            nn.Conv2d(256, 256, kernel_size=(1, 7), padding=(0, 3)),
            nn.Conv2d(256, 320, kernel_size=(7, 1), padding=(3, 0)),
            nn.Conv2d(320, 320, kernel_size=3, stride=2)
        )

        self.branch3 = nn.MaxPool2d(kernel_size=3, stride=2)

      def forward(self, x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        out = torch.cat([b1, b2, b3], dim=1)
        return out

    class InceptionC(nn.Module):
      def __init__(self, in_channels):
        super(InceptionC, self).__init__()

        self.branch1 = nn.Conv2d(in_channels, 256, kernel_size=1)

        self.branch2_1 = nn.Conv2d(in_channels, 384, kernel_size=1)
        self.branch2_2 = nn.Conv2d(384, 256, kernel_size=(1, 3), padding=(0, 1))
        self.branch2_3 = nn.Conv2d(384, 256, kernel_size=(3, 1), padding=(1, 0))

        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, 384, kernel_size=1),
            nn.Conv2d(384, 448, kernel_size=3, padding=1),
            nn.Conv2d(448, 512, kernel_size=3, padding=1)
        )

        self.branch3_2a = nn.Conv2d(512, 256, kernel_size=(1, 3), padding=(0, 1))
        self.branch3_2b = nn.Conv2d(512, 256, kernel_size=(3, 1), padding=(1, 0))

        self.branch4 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, 256, kernel_size=1)
        )

      def forward(self, x):
        b1 = self.branch1(x)

        b2 = self.branch2_1(x)
        b2a = self.branch2_2(b2)
        b2b = self.branch2_3(b2)
        b2 = torch.cat([b2a, b2b], dim=1)

        b3 = self.branch3(x)
        b3a = self.branch3_2a(b3)
        b3b = self.branch3_2b(b3)
        b3 = torch.cat([b3a, b3b], dim=1)

        b4 = self.branch4(x)

        out = torch.cat([b1, b2, b3, b4], dim=1)
        return out

    class InceptionV4(nn.Module):
      def __init__(self, num_classes=2):
        super(InceptionV4, self).__init__()

        self.stem = Stem()

        self.InceptionA = nn.Sequential(
            InceptionA(384),
            InceptionA(384),
            InceptionA(384),
            InceptionA(384)
        )

        self.reductionA = ReductionA(384)

        self.InceptionB = nn.Sequential(
            InceptionB(1024),
            InceptionB(1024),
            InceptionB(1024),
            InceptionB(1024),
            InceptionB(1024),
            InceptionB(1024),
            InceptionB(1024),
        )

        self.reductionB = ReductionB(1024)

        self.InceptionC = nn.Sequential(
            InceptionC(1536),
            InceptionC(1536),
            InceptionC(1536)
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        if use_dropout:
          self.head_drop = nn.Dropout(p=0.2)
          self.classif = nn.Sequential(
              nn.Linear(1536, 256),
              nn.ReLU(),
              nn.Dropout(0.2),
              nn.Linear(256, num_classes),
          )
        else:
          self.head_drop = nn.Dropout(p=0.0)
          self.classif = nn.Sequential(
              nn.Linear(1536, 256),
              nn.ReLU(),
              nn.Linear(256, num_classes),
          )

      def forward(self, x):
        x = self.stem(x)
        x = self.InceptionA(x)
        x = self.reductionA(x)
        x = self.InceptionB(x)
        x = self.reductionB(x)
        x = self.InceptionC(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.head_drop(x)
        x = self.classif(x)
        return x

    return InceptionV4().to(device)

# 모델 저장 시에는 다음처럼 state_dict만 저장하세요:
# torch.save(model.state_dict(), path)

# 모델 로딩 시에는 다음처럼 사용하세요:
# model = create_model()
# model.load_state_dict(torch.load(path))
# model.eval()


In [ ]:
''' 학습 모델 '''
# =============================
# 목적 : 모델 학습
# Early Stopping : val loss가 5번 이상 개선이 안될 경우, 종료
# =============================
def TrainModel(model, criterion, optimizer, scheduler, trainLoader, valLoader, numEpochs=20, patience=5):
    # Early Stopping 관련 변수
    # loss 기준값을 최댓값으로 설정
    best_val_loss = float('inf')
    # early stopping counter 0으로 설정
    counter = 0
    # best model 저장 경로 설정
    best_model_path = f"/content/drive/MyDrive/model/Inception v4 초기학습/{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model.pth"

    # Backbone 동결 상태에서 fc layer만 학습
    for epoch in range(numEpochs):
        # Train
        # 모델을 학습 모드로 설정
        model.train()
        runningLoss = 0.0
        runningCorrects = 0

        # 훈련 데이터 반복 학습
        for inputs, labels in trainLoader:
            # 이미지와 라벨(클래스)
            inputs, labels = inputs.to(device), labels.to(device)

            # 기존 그래디언트 초기화
            optimizer.zero_grad()
            # 모델 forward pass
            outputs = model(inputs)
            # 손실함수 계산 (출력, 정답)
            loss = criterion(outputs, labels)
            # 역전파 학습
            loss.backward()
            # 파라미터 업데이트
            optimizer.step()

            # 예측값 계산
            _, preds = torch.max(outputs, 1)
            # 손실 합산
            runningLoss += loss.item() * inputs.size(0)
            # 정확도 합산
            runningCorrects += torch.sum(preds == labels.data)

        # 반복 횟수 단위로 Train 평균 손실 및 정확도 계산
        epochLoss = runningLoss / len(trainLoader.dataset)
        epochAcc = runningCorrects.double() / len(trainLoader.dataset)
        print(f"Epoch {epoch+1}/{numEpochs} - Train Loss: {epochLoss:.4f} Acc: {epochAcc:.4f}")

        # Validation
        # 모델을 평가 모드로 설정
        model.eval()
        valLoss = 0.0
        valCorrects = 0

        # 그래디언트 비활성화
        with torch.no_grad():
            for inputs, labels in valLoader:
                # 이미지와 라벨(클래스)
                inputs, labels = inputs.to(device), labels.to(device)
                # 모델 forward pass
                outputs = model(inputs)
                # 손실함수 계산 (출력, 정답)
                loss = criterion(outputs, labels)

                # 예측값 계산
                _, preds = torch.max(outputs, 1)
                # 손실 합산
                valLoss += loss.item() * inputs.size(0)
                # 정확도 합산
                valCorrects += torch.sum(preds == labels.data)

        # 반복 횟수 단위로 Validation 평균 손실 및 정확도 계산
        valLoss /= len(valLoader.dataset)
        val_acc = valCorrects.double() / len(valLoader.dataset)
        print(f"Validation Loss: {valLoss:.4f} Acc: {val_acc:.4f}")

        # Scheduler: Validation Loss 기반으로 학습률 조정
        scheduler.step(valLoss)

        # Early Stopping
        # 개선이 되면
        if valLoss < best_val_loss:
            best_val_loss = valLoss
            counter = 0
            torch.save(model, best_model_path)
            print(f"Renewal best model: {best_val_loss:.4f}")
        # 개선이 안되면
        else:
            counter += 1
            print(f"No improvement in {counter}/{patience} epochs")
            if counter >= patience:
                break

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
import numpy as np
import torch.nn.functional as F

def TestModel(model, testLoader):
    all_preds = []
    all_probs = []  # AUC 계산용 softmax 확률
    all_labels = []

    testCorrects = 0
    testLoss = 0.0

    criterion = nn.CrossEntropyLoss()
    model.eval()

    with torch.no_grad():
        for inputs, labels in testLoader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 예측 클래스
            _, preds = torch.max(outputs, 1)
            # softmax 확률 (클래스 1의 확률만)
            probs = F.softmax(outputs, dim=1)[:, 1]

            testLoss += loss.item() * inputs.size(0)
            testCorrects += torch.sum(preds == labels.data)

            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 평균 계산
    testLoss /= len(testLoader.dataset)
    testAcc = testCorrects.double() / len(testLoader.dataset)
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    auc = roc_auc_score(all_labels, all_probs)

    print(f"Test Accuracy  : {testAcc:.4f}")
    print(f"Test Loss      : {testLoss:.4f}")
    print(f"ROC-AUC        : {auc:.4f}")
    print(f"Precision      : {precision:.4f}")
    print(f"Recall         : {recall:.4f}")
    print(f"F1-macro       : {f1:.4f}")

    return testAcc.item(), testLoss, auc, precision, recall, f1

In [ ]:
def initialize_weights(model):
    for name, m in model.named_modules():
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Linear):
            if 'classif' in name:
                nn.init.normal_(m.weight, mean=0.0, std=0.5)  # 💥 폭넓게
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.2)
            else:
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.1)

In [ ]:
model = create_model()
x = torch.randn(4, 3, 299, 299).to(device)
y = model(x)
print("FC output shape:", y.shape)
print("Logits mean/std:", y.mean().item(), y.std().item())


FC output shape: torch.Size([4, 2])
Logits mean/std: 0.019397489726543427 0.035522330552339554


In [ ]:
import torch.nn as nn
import torch.nn.init as init

def init_weights_he(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            init.zeros_(m.bias)

## 모델 학습 및 테스트

In [ ]:
for i in range(0,2):
  # 모델 불러오기
  model = create_model()
  model.apply(init_weights_he)

  # CrossEntropyLoss로 손실 함수 설정
  criterion = nn.CrossEntropyLoss()
  # Adam으로 옵티마이저로 사용 (filter를 통해 require_grad = True, 동결되지 않은 것만 업데이트)
  # weight_decay = 1e-4 : L2 정규화 적용으로 과적합 방지
  if use_l2:
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
  else:
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

  # 학습률 스케줄러 설정
  # 검증 손실이 줄어들지 않으면 learning rate 줄임 (수렴 및 안정화)
  # min 모드 : 손실이 최소화되지 않으면 작동 / factor : 학습률을 x배 줄임 / patience : x번 학습동안 개선되지 않으면 발동 / verbose : 메시지 출력
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

  # Train 모델 실행
  TrainModel(model, criterion, optimizer, scheduler, trainLoader, valLoader, numEpochs=20, patience=5)

  os.rename(f"/content/drive/MyDrive/model/Inception v4 초기학습/{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model.pth", f"/content/drive/MyDrive/model/Inception v4 초기학습/{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model_{i+1}.pth")

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20 - Train Loss: 18758691966393.8828 Acc: 0.5207
Validation Loss: 412088465931.9070 Acc: 0.6492


AttributeError: Can't pickle local object 'create_model.<locals>.InceptionV4'

In [ ]:
print(f"{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model.pth")
for i in range(2):
  best_model_path = f"/content/drive/MyDrive/model/Inception v4 초기학습/{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model_{i+1}.pth"
  model = torch.load(best_model_path, weights_only=False)
  model.to(device)

  # 테스트
  result = TestModel(model, testLoader)
  print()

구버전

In [ ]:
''' 학습 모델 '''
# =============================
# 목적 : 모델 학습
# 단일 단계 학습 (Inception v4: 사전학습 X)
# Early Stopping : val loss가 6번 이상 개선이 안될 경우, 종료
# =============================
def TrainModel(model, criterion, optimizer, scheduler, trainLoader, valLoader, numEpochs=12, patience=6):
    # Early Stopping 관련 변수
    best_val_loss = float('inf')
    counter = 0
    best_model_path = f"/content/drive/MyDrive/model/Inception v4 초기학습/{use_background}{use_agumentation}{use_dropout}{use_l2}_best_model.pth"

    for epoch in range(numEpochs):
        # Train
        model.train()
        runningLoss = 0.0
        runningCorrects = 0

        for inputs, labels in trainLoader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            runningLoss += loss.item() * inputs.size(0)
            runningCorrects += torch.sum(preds == labels.data)

        epochLoss = runningLoss / len(trainLoader.dataset)
        epochAcc = runningCorrects.double() / len(trainLoader.dataset)
        print(f"Epoch {epoch+1}/{numEpochs} - Train Loss: {epochLoss:.4f} Acc: {epochAcc:.4f}")

        # Validation
        model.eval()
        valLoss = 0.0
        valCorrects = 0

        with torch.no_grad():
            for inputs, labels in valLoader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)
                valLoss += loss.item() * inputs.size(0)
                valCorrects += torch.sum(preds == labels.data)

        valLoss /= len(valLoader.dataset)
        val_acc = valCorrects.double() / len(valLoader.dataset)
        print(f"Validation Loss: {valLoss:.4f} Acc: {val_acc:.4f}")

        #scheduler.step(valLoss) # 확인 필요
        scheduler.step()
        if valLoss < best_val_loss:
            best_val_loss = valLoss
            counter = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"Renewal best model: {best_val_loss:.4f}")
        else:
            counter += 1
            print(f"No improvement in {counter}/{patience} epochs")
            if counter >= patience:
                break

In [ ]:
# 모델 불러오기
model = create_model()

# 가중치 초기화 적용
initialize_weights(model)
print(type(model.stem.conv1.weight))  # Parameter
print(model.stem.conv1.weight.mean(), model.stem.conv1.weight.std())

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# 옵티마이저 설정 (프리징 없음 → 전체 파라미터 사용)
if use_l2:
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4) # lr => 0.001?
else:
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Cosine Annealing 스케줄러 설정 (warm-up 없이 단일 단계) # 확인 필요 동일한 방식으로 해야하지 않을까...!
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=20,
    eta_min=1e-6
)

# 모델 학습 실행
TrainModel(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    trainLoader=trainLoader,
    valLoader=valLoader,
    numEpochs=20,
    patience=5
)


<class 'torch.nn.parameter.Parameter'>
tensor(1.6994e-05, device='cuda:0', grad_fn=<MeanBackward0>) tensor(0.0798, device='cuda:0', grad_fn=<StdBackward0>)
Training full model (no freezing, since pretrained weights are not used)
Epoch 1/20 - Train Loss: 11.6860 Acc: 0.5956
Validation Loss: 0.6984 Acc: 0.6667
Saved best model with Val Loss: 0.6984
Epoch 2/20 - Train Loss: 0.6851 Acc: 0.6680
Validation Loss: 0.7240 Acc: 0.6531
No improvement in 1/5 epochs
Epoch 3/20 - Train Loss: 0.6393 Acc: 0.6673
Validation Loss: 0.6054 Acc: 0.7074
Saved best model with Val Loss: 0.6054
Epoch 4/20 - Train Loss: 0.6132 Acc: 0.6970
Validation Loss: 0.6467 Acc: 0.6938
No improvement in 1/5 epochs
Epoch 5/20 - Train Loss: 0.5894 Acc: 0.6932
Validation Loss: 0.5805 Acc: 0.7267
Saved best model with Val Loss: 0.5805
Epoch 6/20 - Train Loss: 0.6000 Acc: 0.6990
Validation Loss: 0.5881 Acc: 0.7035
No improvement in 1/5 epochs
Epoch 7/20 - Train Loss: 0.5624 Acc: 0.7151
Validation Loss: 0.5844 Acc: 0.7093
No imp

In [ ]:
# 1. 모델 구조를 다시 정의
model = create_model()

# 2. 저장된 가중치 로드
best_model_path = f"/content/drive/MyDrive/model/{use_background}{use_agumentation}{use_dropout}{use_freezing}{use_l2}_best_model_초기학습.pth"
state_dict = torch.load(best_model_path)

# 3. 모델에 가중치 적용
model.load_state_dict(state_dict)

# 4. 디바이스로 이동
model.to(device)

# 5. 평가 모드 전환 (꼭!)
model.eval()

# 6. 테스트 실행
result = TestModel(model, testLoader)


Test Accuracy  : 0.8101
Test Loss      : 0.3916
ROC-AUC        : 0.9080
Precision      : 0.8135
Recall         : 0.8109
F1-macro       : 0.8098
